In [63]:
!pip install eli5

In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
import eli5
from eli5.sklearn import PermutationImportance
from ast import literal_eval
from tqdm import tqdm_notebook

In [65]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [66]:
ls data

men_shoes.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory= False)

In [68]:
df.columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18280 entries, 0 to 18279
Data columns (total 48 columns):
id                     18280 non-null object
asins                  2161 non-null object
brand                  18263 non-null object
categories             18280 non-null object
colors                 10344 non-null object
count                  0 non-null float64
dateadded              18280 non-null object
dateupdated            18280 non-null object
descriptions           9704 non-null object
dimension              2968 non-null object
ean                    9690 non-null float64
features               13299 non-null object
flavors                0 non-null float64
imageurls              17255 non-null object
isbn                   0 non-null float64
keys                   18280 non-null object
manufacturer           6296 non-null object
manufacturernumber     14309 non-null object
merchants              13018 non-null object
name                   18280 non-null object
pric

In [0]:
def run_model (feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring = 'neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [70]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [71]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [72]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [73]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'
literal_eval(str_dict)



[{'key': 'Gender', 'value': ['Men']},
 {'key': 'Shoe Size', 'value': ['M']},
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
 {'key': 'Color', 'value': ['Multicolor']},
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
 {'key': 'Brand', 'value': ['Josmo']}]

In [0]:
def parse_features(x):  
  output_dict = {}
  if str(x) == 'nan': return output_dict
  features = literal_eval(x.replace('\\"','"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value
  
  return output_dict

df['features_parsed'] = df['features'].map(parse_features)


In [75]:
df.features_parsed.head()

0    {'gender': 'men', 'shoe size': 'm', 'shoe cate...
1    {'gender': 'men', 'shoe size': 'm', 'shoe cate...
2    {'gender': 'men', 'color': 'black', 'shipping ...
3    {'gender': 'men', 'color': 'black', 'shipping ...
4    {'gender': 'men', 'color': 'black', 'shipping ...
Name: features_parsed, dtype: object

In [76]:
keys = set()
df['features_parsed'].map( lambda x: keys.update(x.keys()) )
len(keys)

476

In [77]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys): 
  # print(key)

  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats [key] if key in feats else np.nan)

In [78]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_clothing size type', 'feat_shape', 'feat_fits styles', 'feat_fit',
       'feat_domestic shipping', 'feat_style:', 'feat_shipping to usa',
       'feat_item type', 'feat_bag size range',
       'feat_safety apparel product type'],
      dtype='object', length=526)

In [57]:
df.brand

0                      Josmo
1                      Josmo
2        SERVUS BY HONEYWELL
3        SERVUS BY HONEYWELL
4        SERVUS BY HONEYWELL
                ...         
18275             Xray, XRay
18276                   Fila
18277            Seek Optics
18278            Seek Optics
18279            Seek Optics
Name: brand, Length: 18280, dtype: object

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100


In [91]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_color_cat'] = df['feat_color'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].map(lambda x: str(x).lower()).factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_style_cat'] = df['feat_style'].map(lambda x: str(x).lower()).factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]


In [105]:
df['brand'] = df['brand'].map(lambda x: str(x).lower() )
# df[df.brand == df.feat_brand ].shape
df[ df.brand != df.feat_brand ] [ [ 'brand', 'feat_brand'] ].head()

,brand,feat_brand
21,rubies,generic
22,rubies,generic
23,rubies,generic
24,unbranded,NaN
31,american fighter,NaN


In [0]:
feaats = 

In [107]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-57.276464143646024, 4.157717303596153)

In [132]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'feat_shoe category',
 'feat_certifications and listings',
 'feat_location - country',
 'feat_recommended location',
 'feat_clothing category',
 'feat_multi pack indicator',
 'feat_catalog',
 'feat_fabrication',
 'feat_location - city/state',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_stylenumber_cat',
 'feat_lens type_cat',
 'feat_fastener_cat',
 'feat_has mercury_cat',
 'feat_number of heat settings_cat',
 'feat_fabric_cat',
 'feat_shoe category_cat',
 'feat_bezel_cat',
 'feat_charge time (minutes)_cat',
 'feat_clothing size_cat',
 'feat_pant style_cat',
 'feat_product id_cat',
 'feat_protection_cat',
 'feat_character_cat',
 'feat_motion control_cat',
 'feat_country of manufacture_cat',
 'feat_date first available at amazon.ca_cat',
 'feat_bridge width_cat',
 'feat_construction_cat',
 'feat_leather grade_cat',
 'feat_batteries required?

In [144]:
feats = ['brand_cat','feat_brand_cat','feat_shape_cat','feat_gender_cat','feat_material_cat', 'feat_style_cat', 'feat_sport_cat']
# feats += feats_cat
# feats = list(set(feats))

model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)

(-57.085652189922314, 4.311091458574312)

In [145]:
X = df[ feats ].values
y = df[ 'prices_amountmin' ].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)
print(result)
perm = PermutationImportance(m, random_state = 1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)



(-57.06008093188029, 4.3398821030333)


Weight,Feature
0.2638 ± 0.0108,brand_cat
0.1069 ± 0.0144,feat_material_cat
0.0224 ± 0.0033,feat_gender_cat
0.0217 ± 0.0004,feat_brand_cat
0.0134 ± 0.0011,feat_shape_cat
0.0043 ± 0.0007,feat_style_cat
0.0003 ± 0.0000,feat_sport_cat


In [126]:
df[df['brand'] == 'nike'].features_parsed.sample(7).values

array([{'gender': 'boys', 'shoe category': 'boys shoes', 'color': 'lyon blue/metallic silver/blue lagoon/white', 'casual & dress shoe style': 'athletic', 'manufacturer part number': '644431402', 'brand': 'nike', 'shoe width': 'medium'},
       {'manufacturer part number': '312618311', 'gender': 'men', 'shoe category': 'mens shoes', 'brand': 'nike', 'color': 'venom green/white', 'casual & dress shoe style': 'sandals', 'shoe width': 'medium (d, m)'},
       {'manufacturer part number': '823581-012', 'season': 'all-season', 'gender': 'men', 'material': 'synthetic', 'shoe category': 'athletic & sneakers', 'brand': 'nike', 'age group': 'adult', 'color': 'black'},
       {'material': 'leather', 'gender': 'men', 'size': '12', 'color': 'white', 'model': '378336 120', 'manufacturer part number': '378336 120', 'brand': 'nike', 'age group': 'adult'},
       {'sport': 'soccer', 'condition': 'new without box', 'type': 'cleats'},
       {'season': 'all-season', 'material': 'synthetic', 'gender': 'me

In [140]:
df.weight

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
18275    NaN
18276    NaN
18277    NaN
18278    NaN
18279    NaN
Name: weight, Length: 18280, dtype: object

In [146]:
pwd

'/content/drive/My Drive/Colab Notebooks/dw_matrix'

In [147]:
ls

data/  HelloGithub.ipynb  LICENSE  matrix_one/  README.md


In [0]:
! git add matrixone/day5.ipynb